In [ ]:
#AI Augmented Forecast Risk & Confidence Scoring

#This project demonstrates how AI can be responsibly applied to enterprise sales analytics to assess forecast confidence and highlight pipeline risk for senior leadership.

In [2]:
#Importing key packages & data

import pandas as pd
from datetime import datetime

df = pd.read_csv(r"", parse_dates=["close_date", "created_date", "last_modified_date"])

today = pd.Timestamp(datetime.today().date())
df["days_open"] = (today - df["created_date"]).dt.days
df["days_since_update"] = (today - df["last_modified_date"]).dt.days

df.head()

C:\Users\CHAN9473\Anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '31/03/2026' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\CHAN9473\Anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '15/04/2026' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\CHAN9473\Anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '28/02/2026' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\CHAN9473\Anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '30/05/2026' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\CHAN947

,opportunity_id,account_name,deal_value,stage,probability,forecast_category,days_in_stage,close_date,created_date,last_modified_date,owner_role,industry,days_open,days_since_update
0,OPP-001,Apex Utilities,750000,Negotiation,40,Omitted,68,2026-03-31,2025-01-06,2026-10-01,Sales Manager,Utilities,404,-229
1,OPP-002,Orion Defence,2200000,Proposal,55,Best Case,45,2026-04-15,2025-10-07,2026-01-25,Account Executive,Defence,130,20
2,OPP-003,Nova Tech,180000,Qualification,20,Omitted,30,2026-01-06,2025-01-12,2026-01-02,Sales Rep,Technology,398,43
3,OPP-004,Helios Energy,1250000,Negotiation,70,Commit,82,2026-02-28,2025-05-15,2026-05-01,Sales Director,Energy,275,-76
4,OPP-005,Atlas Secure,3100000,Proposal,50,Best Case,95,2026-05-30,2025-01-04,2026-01-15,Sales Director,Defence,406,30


In [3]:
#Defining the risk score logic

def calculate_risk(row):
    risk = 0

    if row["deal_value"] > 1000000 and row["probability"] < 60:
        risk += 30

    if row["days_in_stage"] > 90:
        risk += 25

    if row["forecast_category"] == "Omitted":
        risk += 20

    if row["days_since_update"] > 30:
        risk += 15

    if row["stage"] in ["Proposal", "Negotiation"] and row["probability"] < 50:
        risk += 10

    return min(risk, 100)

df["risk_score"] = df.apply(calculate_risk, axis=1)

df["confidence_band"] = pd.cut(
    df["risk_score"],
    bins=[-1, 30, 60, 100],
    labels=["High Confidence", "Medium Confidence", "Low Confidence"]
)

df[["opportunity_id", "deal_value", "risk_score", "confidence_band"]]

,opportunity_id,deal_value,risk_score,confidence_band
0,OPP-001,750000,30,High Confidence
1,OPP-002,2200000,30,High Confidence
2,OPP-003,180000,35,Medium Confidence
3,OPP-004,1250000,0,High Confidence
4,OPP-005,3100000,55,Medium Confidence
5,OPP-006,420000,0,High Confidence
6,OPP-007,980000,70,Low Confidence
7,OPP-008,560000,0,High Confidence
8,OPP-009,4100000,25,High Confidence
9,OPP-010,890000,45,Medium Confidence


In [4]:
#Creating the data insights

summary = {
    "total_pipeline_value": df["deal_value"].sum(),
    "high_risk_pipeline_value": df[df["confidence_band"] == "Low Confidence"]["deal_value"].sum(),
    "percent_pipeline_high_risk": round(
        df[df["confidence_band"] == "Low Confidence"]["deal_value"].sum() / df["deal_value"].sum() * 100, 1
    )
}

high_risk_deals = df[df["confidence_band"] == "Low Confidence"][
    ["opportunity_id", "account_name", "deal_value", "stage", "risk_score"]
].sort_values(by="risk_score", ascending=False)

summary, high_risk_deals

({'total_pipeline_value': 14430000,
  'high_risk_pipeline_value': 980000,
  'percent_pipeline_high_risk': 6.8},
   opportunity_id    account_name  deal_value        stage  risk_score
 6        OPP-007  Vertex Systems      980000  Negotiation          70)

In [5]:
#Building the prompt template using artificial intelligence

prompt = f"""
You are a commercial analytics advisor.

Pipeline overview:
- Total pipeline value: £{summary['total_pipeline_value']:,}
- High-risk pipeline value: £{summary['high_risk_pipeline_value']:,}
- Percentage of pipeline at high risk: {summary['percent_pipeline_high_risk']}%

High-risk opportunities:
{high_risk_deals.to_string(index=False)}

Produce an executive briefing covering:
1. Key forecast risks
2. Where leadership attention is required
3. Suggested actions to improve forecast confidence
"""

print(prompt)


You are a commercial analytics advisor.

Pipeline overview:
- Total pipeline value: £14,430,000
- High-risk pipeline value: £980,000
- Percentage of pipeline at high risk: 6.8%

High-risk opportunities:
opportunity_id   account_name  deal_value       stage  risk_score
       OPP-007 Vertex Systems      980000 Negotiation          70

Produce an executive briefing covering:
1. Key forecast risks
2. Where leadership attention is required
3. Suggested actions to improve forecast confidence



In [7]:
#Saving the executive briefing as an output

import os

os.makedirs("outputs", exist_ok=True)

with open("", "w") as file:
    file.write(prompt)

print("Executive summary prompt saved to outputs/exec_summary.txt")

Executive summary prompt saved to outputs/exec_summary.txt
